# Nornir Tutorial

## Initializing Nornir

In [21]:
from nornir import InitNornir

nr = InitNornir(config_file="config.yaml")

# nr = InitNornir(
#     runner={
#         "plugin": "threaded",
#         "options": {
#             "num_workers": 100,
#         },
#     },
#     inventory={
#         "plugin": "SimpleInventory",
#         "options": {
#             "host_file": "inventory/hosts.yaml",
#             "group_file": "inventory/groups.yaml"
#         },
#     },
# )

# nr = InitNornir(
#     config_file="config.yaml",
#     runner={
#         "plugin": "threaded",
#         "options": {
#             "num_workers": 50,
#         },
#     },
# )

In [22]:
nr.config.runner.options

{'num_workers': 100}

## Inventory 

In [23]:
from nornir.core.inventory import Host
import json
print(json.dumps(Host.schema(), indent=4))


{
    "name": "str",
    "connection_options": {
        "$connection_type": {
            "extras": {
                "$key": "$value"
            },
            "hostname": "str",
            "port": "int",
            "username": "str",
            "password": "str",
            "platform": "str"
        }
    },
    "groups": [
        "$group_name"
    ],
    "data": {
        "$key": "$value"
    },
    "hostname": "str",
    "port": "int",
    "username": "str",
    "password": "str",
    "platform": "str"
}


In [24]:
nr.inventory.hosts

{'host1.cmh': Host: host1.cmh,
 'host2.cmh': Host: host2.cmh,
 'spine00.cmh': Host: spine00.cmh,
 'spine01.cmh': Host: spine01.cmh,
 'leaf00.cmh': Host: leaf00.cmh,
 'leaf01.cmh': Host: leaf01.cmh,
 'host1.bma': Host: host1.bma,
 'host2.bma': Host: host2.bma,
 'spine00.bma': Host: spine00.bma,
 'spine01.bma': Host: spine01.bma,
 'leaf00.bma': Host: leaf00.bma,
 'leaf01.bma': Host: leaf01.bma}

In [25]:
nr.inventory.groups

{'global': Group: global,
 'eu': Group: eu,
 'bma': Group: bma,
 'cmh': Group: cmh}

In [26]:
nr.inventory.hosts["leaf01.bma"]

Host: leaf01.bma

In [27]:
nr.inventory.groups

{'global': Group: global,
 'eu': Group: eu,
 'bma': Group: bma,
 'cmh': Group: cmh}

In [28]:
host = nr.inventory.hosts["leaf01.bma"]
host.keys()

dict_keys(['site', 'role', 'type', 'asn', 'domain'])

In [29]:
host["site"]

'bma'

## Filter

In [30]:
nr.filter(site="cmh").inventory.hosts.keys()

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'spine01.cmh', 'leaf00.cmh', 'leaf01.cmh'])

In [31]:
nr.filter(site="cmh", role="spine").inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh'])

In [32]:
nr.filter(site="cmh").filter(role="spine").inventory.hosts.keys()

dict_keys(['spine00.cmh', 'spine01.cmh'])

In [33]:
from nornir.core.filter import F

# hosts in group cmh
cmh = nr.filter(F(groups__contains="cmh"))
print(cmh.inventory.hosts.keys())

dict_keys(['host1.cmh', 'host2.cmh', 'spine00.cmh', 'spine01.cmh', 'leaf00.cmh', 'leaf01.cmh'])


## Tasks

In [42]:
from nornir.core.task import Task, Result
from nornir_utils.plugins.functions import print_result

def hello_world(task: Task) -> Result:
    return Result(
        host=task.host,
        result=f"{task.host.name} says hello world!"
    )
result = nr.run(task=hello_world)
print_result(result)


hello_world*********************************************************************
* host1.bma ** changed : False *************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host1.bma says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host1.cmh ** changed : False *************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host1.cmh says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host2.bma ** changed : False *************************************************
vvvv hello_world ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
host2.bma says hello world!
^^^^ END hello_world ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv hello

In [43]:
def say(task: Task, text: str) -> Result:
    return Result(
        host=task.host,
        result=f"{task.host.name} says {text}"
    )

result = nr.run(
    name="Saying goodbye in a very friendly manner",
    task=say,
    text="buhbye!"
)
print_result(result)

Saying goodbye in a very friendly manner****************************************
* host1.bma ** changed : False *************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
host1.bma says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host1.cmh ** changed : False *************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
host1.cmh says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host2.bma ** changed : False *************************************************
vvvv Saying goodbye in a very friendly manner ** changed : False vvvvvvvvvvvvvvv INFO
host2.bma says buhbye!
^^^^ END Saying goodbye in a very friendly manner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host2.cmh ** changed : False *************************************************
vvvv Saying goodbye in a 

## Grouping tasks

In [44]:
def count(task: Task, number: int) -> Result:
    return Result(
        host=task.host,
        result=f"{[n for n in range(0, number)]}"
    )

def greet_and_count(task: Task, number: int) -> Result:
    task.run(
        name="Greeting is the polite thing to do",
        task=say,
        text="hi!",
    )

    task.run(
        name="Counting beans",
        task=count,
        number=number,
    )
    task.run(
        name="We should say bye too",
        task=say,
        text="bye!",
    )

    # let's inform if we counted even or odd times
    even_or_odds = "even" if number % 2 == 1 else "odd"
    return Result(
        host=task.host,
        result=f"{task.host} counted {even_or_odds} times!",
    )

result = nr.run(
    name="Counting to 5 while being very polite",
    task=greet_and_count,
    number=5,
)
print_result(result)

Counting to 5 while being very polite*******************************************
* host1.bma ** changed : False *************************************************
vvvv Counting to 5 while being very polite ** changed : False vvvvvvvvvvvvvvvvvv INFO
host1.bma counted even times!
---- Greeting is the polite thing to do ** changed : False --------------------- INFO
host1.bma says hi!
---- Counting beans ** changed : False ----------------------------------------- INFO
[0, 1, 2, 3, 4]
---- We should say bye too ** changed : False ---------------------------------- INFO
host1.bma says bye!
^^^^ END Counting to 5 while being very polite ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* host1.cmh ** changed : False *************************************************
vvvv Counting to 5 while being very polite ** changed : False vvvvvvvvvvvvvvvvvv INFO
host1.cmh counted even times!
---- Greeting is the polite thing to do ** changed : False --------------------- INFO
host1.cmh says hi!
---- Counting beans ** cha